In [1]:
import os
import time
import pickle

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneOut

In [4]:
feature_path = os.path.abspath('../../features/inception')
model_save_path = os.path.abspath('../../models/inception_svm')
leave_one_out_validation = False

In [5]:
#Storing the extracted features
no_ball = np.load(f'{feature_path}/model2_inception_features_no_ball.npy')
out = np.load(f'{feature_path}/model2_inception_features_out.npy')
sixes = np.load(f'{feature_path}/model2_inception_features_sixes.npy')
wide = np.load(f'{feature_path}/model2_inception_features_wide.npy')
no_action = np.load(f'{feature_path}/model2_inception_features_no_action.npy')

In [6]:
X_data = np.append(no_ball, out, axis=0)
X_data = np.append(X_data, sixes, axis=0)
X_data = np.append(X_data, wide, axis=0)
X_data = np.append(X_data, no_action, axis=0)

In [7]:
labels = X_data[:, (X_data.shape[1]-1)]

In [8]:
train_data = X_data[:, 0:(X_data.shape[1]-1)]

In [9]:
#Split the data into 80% training and 20% test data
x_tr, x_ts, y_tr, y_ts = train_test_split(train_data, labels, test_size=0.2, random_state=26)

In [10]:
#Classifier using Linear SVM

clf = LinearSVC(C=10)
start_time = time.time()
clf = clf.fit(x_tr, y_tr)
predictions_tr = (clf.predict(x_ts))

c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [11]:
#10-fold Cross-Validation    
scores = cross_val_score(clf, x_tr, y_tr, cv=10)
test_acc = accuracy_score(y_ts,predictions_tr)

c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\tanmay jain\appdata\local\programs\

In [12]:
print("Training Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Test Accuracy: %0.4f" % test_acc)
print("--- %s seconds ---" % (time.time() - start_time))

Training Accuracy: 0.7579 (+/- 0.23)
Test Accuracy: 0.7619
--- 33.5022714138031 seconds ---


In [13]:
##Leave One Out Validation
if leave_one_out_validation:
   loo_train_acc = []
   loo = LeaveOneOut()
   for train_index, test_index in loo.split(x_tr):
      X_train, X_test = x_tr[train_index], x_tr[test_index]
      y_train, y_test = y_tr[train_index], y_tr[test_index]
      clf = clf.fit(X_train,y_train)
      predictions = (clf.predict(X_test))
      loo_train_acc.append(accuracy_score(y_test,predictions))

   loo_train_accuracy = np.asarray(loo_train_acc)
   print("LOO Accuracy: %0.4f" % loo_train_accuracy.mean())

In [14]:
#Save the model
pickle.dump(clf, open(f'{model_save_path}/model2_inception_svm.sav', 'wb'))